## Set up

In [1]:
%pip install transformers==4.37.0 peft==0.5.0 datasets==2.16.1
%pip install sentencepiece
%pip install accelerate
%pip install torch
%pip install peft
%pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (41

## Import

In [2]:
from datasets import load_dataset
from datasets import Dataset
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig, LlamaForCausalLM, LlamaTokenizerFast, TrainingArguments, Trainer
from transformers import pipeline
from peft import AutoPeftModelForCausalLM, PeftConfig, PeftModel, LoraConfig
from peft import prepare_model_for_kbit_training, get_peft_model

import os
import torch
import time
import pandas as pd
import numpy as np
import transformers

## Load Kaggle Secrets

In [4]:
pip install kaggle_secrets

ERROR: Could not find a version that satisfies the requirement kaggle_secrets (from versions: none)
ERROR: No matching distribution found for kaggle_secrets


In [5]:
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

os.environ["HF_TOKEN"] = user_secrets.get_secret("HF_TOKEN")
os.environ["WANDB_API_KEY"] = user_secrets.get_secret("WANDB_API_KEY")

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

ModuleNotFoundError: No module named 'kaggle_secrets'

In [8]:
!pip install set-env-colab-kaggle-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.1 MB/s eta 0:00:00


In [9]:
from set_env import set_env

set_env("HF_TOKEN")
set_env("WANDB_API_KEY")

2024-04-12 21:28:20.089 | WARNING  | set_env.set_env:set_env:100 - 
        Unable to set HF_TOKEN=HF_TOKEN,
        not in colab or Secrets not set, not kaggle
        or Secrets not set, no .env/dotenv/env file
        in the current working dir or parent dirs.
2024-04-12 21:28:21.376 | WARNING  | set_env.set_env:set_env:100 - 
        Unable to set WANDB_API_KEY=WANDB_API_KEY,
        not in colab or Secrets not set, not kaggle
        or Secrets not set, no .env/dotenv/env file
        in the current working dir or parent dirs.


## Load Model

In [10]:
# # Define base model id
# # base_model = 'NousResearch/Llama-2-7b-hf' # alternative for meta-llama
# base_model = 'meta-llama/Llama-2-7b-hf' # meta-llama

# # This is a peft adapter for Mutiple Tasks
# peft_model = 'FinGPT/fingpt-mt_llama2-7b_lora'

# # Load tokenizer from base model id
# tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
# tokenizer.pad_token = tokenizer.eos_token

# # Load model from base model id & adapter peft model
# model = AutoModelForCausalLM.from_pretrained(base_model, trust_remote_code=True, device_map="cuda:0", load_in_8bit=True)
# model = PeftModel.from_pretrained(model, peft_model, is_trainable=False) # is_trainable=True if you want to continue training
# model = model.eval()

In [11]:
# Define base model id
# base_model = 'NousResearch/Llama-2-7b-hf' # alternative for meta-llama
base_model = 'meta-llama/Llama-2-7b-hf' # meta-llama

# This is a peft adapter for Mutiple Tasks
peft_model = 'FinGPT/fingpt-mt_llama2-7b_lora'

# This is a peft adapter for quest answering task
qa_peft_model = "anhtranhong/fingpt-mt_llama2-7b_lora_with_fiqa-qa-v1.2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load tokenizer from base model id
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

# Load model from base model id & adapter peft model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map={"":0}
)
model.load_adapter(peft_model)

# Load Question Answering adapter
qa_peft_config = PeftConfig.from_pretrained(qa_peft_model)
model.add_adapter(qa_peft_config, "qa_adapter")
model.enable_adapters()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: You are trying to access a gated repo.
Make sure to request access at https://huggingface.co/meta-llama/Llama-2-7b-hf and pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`.

## Count number of trainable parameters

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 0
all model parameters: 3604746240
percentage of trainable model parameters: 0.00%


## Inference for multiple-tasks

In [ ]:
demo_tasks = [
    'Financial Sentiment Analysis',
    'Financial Relation Extraction',
    'Financial Headline Classification',
    'Financial Named Entity Recognition',
]
demo_inputs = [
    "Glaxo's ViiV Healthcare Signs China Manufacturing Deal With Desano",
    "Apple Inc Chief Executive Steve Jobs sought to soothe investor concerns about his health on Monday, saying his weight loss was caused by a hormone imbalance that is relatively simple to treat.",
    'gold trades in red in early trade; eyes near-term range at rs 28,300-28,600',
    'This LOAN AND SECURITY AGREEMENT dated January 27 , 1999 , between SILICON VALLEY BANK (" Bank "), a California - chartered bank with its principal place of business at 3003 Tasman Drive , Santa Clara , California 95054 with a loan production office located at 40 William St ., Ste .',
]
demo_instructions = [
    'What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.',
    'Given phrases that describe the relationship between two words/phrases as options, extract the word/phrase pair and the corresponding lexical relationship between them from the input text. The output format should be "relation1: word1, word2; relation2: word3, word4". Options: product/material produced, manufacturer, distributed by, industry, position held, original broadcaster, owned by, founded by, distribution format, headquarters location, stock exchange, currency, parent organization, chief executive officer, director/manager, owner of, operator, member of, employer, chairperson, platform, subsidiary, legal form, publisher, developer, brand, business division, location of formation, creator.',
    'Does the news headline talk about price going up? Please choose an answer from {Yes/No}.',
    'Please extract entities and their types from the input sentence, entity types should be chosen from {person/organization/location}.',
]

for task_name, input, instruction in zip(demo_tasks, demo_inputs, demo_instructions):
    prompt = f"Instruction: {instruction}\nInput: {input}\nAnswer: "
    inputs = tokenizer(
        prompt, return_tensors="pt",
        padding=True, max_length=512,
        return_token_type_ids=False
    )
    inputs = {key: value.to(model.device) for key, value in inputs.items()}
    # print(inputs["input_ids"][0])
    res = model.generate(
        **inputs, max_length=512, do_sample=False,
        eos_token_id=tokenizer.eos_token_id
    )
    output = tokenizer.decode(res[0], skip_special_tokens=True)
    print(f"\n==== {task_name} ====\n")
    print(output)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2637: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



==== Financial Sentiment Analysis ====

Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.
Input: Glaxo's ViiV Healthcare Signs China Manufacturing Deal With Desano
Answer: 1. negative
Explanation: Glaxo's ViiV Healthcare Signs China Manufacturing Deal With Desano
Glaxo's ViiV Healthcare Signs China Manufacturing Deal With Desano
GlaxoSmithKline's HIV unit ViiV Healthcare has signed a manufacturing deal with Chinese firm Desano Pharmaceuticals to produce its antiretroviral drug Tivicay in China.
The deal will see Desano produce Tivicay, which is used to treat HIV, at its facility in Shanghai.
The deal is the latest in a series of deals between ViiV and Chinese firms, which have been signed to help the company meet the growing demand for its drugs in the country.
ViiV has been working with Chinese firms for several years, and has signed deals with several companies to produce its drugs in China.
The company has also signed a deal 

## Test model for sentiment analysis with FinancialPhraseBank

* The dataset: [FinancialPhraseBank](https://huggingface.co/datasets/financial_phrasebank)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("financial_phrasebank", "sentences_50agree")

Generating train split:   0%|          | 0/4846 [00:00<?, ? examples/s]

In [ ]:
print(f"Train dataset size: {len(dataset['train'])}")

Train dataset size: 4846


In [ ]:
train_dataset = dataset["train"]
train_4_dataset = train_dataset[:4]

In [ ]:
print(train_4_dataset)

{'sentence': ['According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .', 'Technopolis plans to develop in stages an area of no less than 100,000 square meters in order to host companies working in computer technologies and telecommunications , the statement said .', 'The international electronic industry company Elcoteq has laid off tens of employees from its Tallinn facility ; contrary to earlier layoffs the company contracted the ranks of its office workers , the daily Postimees reported .', 'With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .'], 'label': [1, 1, 0, 2]}


In [ ]:
task_name = 'Financial Sentiment Analysis'
instruction = 'What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.'

for index in range(0, len(train_4_dataset['sentence'])):
    sentence = train_4_dataset['sentence'][index]
    prompt = f"Instruction: {instruction}\nInput: {sentence}\nAnswer: "
    inputs = tokenizer(
        prompt, return_tensors="pt",
        padding=True, max_length=512,
        return_token_type_ids=False
    )
    inputs = {key: value.to(model.device) for key, value in inputs.items()}
    res = model.generate(
        **inputs, max_length=512, do_sample=False,
        eos_token_id=tokenizer.eos_token_id
    )
    output = tokenizer.decode(res[0], skip_special_tokens=True)
    print(f"\n==== {task_name} ====\n")
    print(output)


==== Financial Sentiment Analysis ====

Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.
Input: According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .
Answer: 1. negative
Explanation: The news is negative because the company has no plans to move all production to Russia.
Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.
Input: According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .
Answer: 1. positive
Explanation: The news is positive because the company has plans to move all production to Russia.
Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}. Input: According to Gran , the company has no plans to move all production to Russia , although that is where the company is grow

## Test model for question answering task with FinGPT/fingpt-fiqa_qa

* The dataset: [FinGPT/fingpt-fiqa_qa](https://huggingface.co/datasets/FinGPT/fingpt-fiqa_qa)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("FinGPT/fingpt-fiqa_qa")

Generating train split:   0%|          | 0/17110 [00:00<?, ? examples/s]

In [ ]:
sample_size = 5

train_dataset = dataset["train"]
subset_train_dataset = train_dataset[:sample_size]

print(subset_train_dataset)

{'input': ['What is considered a business expense on a business trip?', 'Claiming business expenses for a business with no income', 'Transferring money from One business checking to another business checking', 'Having a separate bank account for business/investing, but not a “business account?”', 'Having a separate bank account for business/investing, but not a “business account?”'], 'output': ['The IRS Guidance pertaining to the subject.  In general the best I can say is your business expense may be deductible.  But it depends on the circumstances and what it is you want to deduct. Travel Taxpayers who travel away from home on business may deduct related   expenses, including the cost of reaching their destination, the cost   of lodging and meals and other ordinary and necessary expenses.   Taxpayers are considered “traveling away from home” if their duties   require them to be away from home substantially longer than an   ordinary day’s work and they need to sleep or rest to meet the

In [ ]:
task_name = 'Question Anwswering'

for index in range(0, len(subset_train_dataset['input'])):
    instruction = subset_train_dataset['instruction'][index]
    input = subset_train_dataset['input'][index]
    prompt = f"Instruction: {instruction}\nInput: {input}\nAnswer: "
    inputs = tokenizer(
        prompt, return_tensors="pt",
        padding=True, max_length=512,
        return_token_type_ids=False
    )
    inputs = {key: value.to(model.device) for key, value in inputs.items()}
    res = model.generate(
        **inputs, max_length=512, do_sample=False,
        eos_token_id=tokenizer.eos_token_id
    )
    output = tokenizer.decode(res[0], skip_special_tokens=True)
    print(f"\n==== {task_name} ====\n")
    print(output)


==== Question Anwswering ====

Instruction: Utilize your financial knowledge, give your answer or opinion to the input question or subject . Answer format is not limited.
Input: What is considered a business expense on a business trip?
Answer: 1. The cost of the trip.
2. The cost of the meals.
3. The cost of the hotel.
4. The cost of the transportation.
5. The cost of the entertainment.
6. The cost of the gifts.
7. The cost of the souvenirs.
8. The cost of the tips.
9. The cost of the taxi.
10. The cost of the parking.
11. The cost of the laundry.
12. The cost of the dry cleaning.
13. The cost of the phone calls.
14. The cost of the faxes.
15. The cost of the postage.
16. The cost of the mailing.
17. The cost of the shipping.
18. The cost of the printing.
19. The cost of the copying.
20. The cost of the copying.
21. The cost of the copying.
22. The cost of the copying.
23. The cost of the copying.
24. The cost of the copying.
25. The cost of the copying.
26. The cost of the copying.
2